In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import os
tf.disable_v2_behavior()
print(np.array([['X11','X12','X13','X14'],['X21','X22','X23','X24']]).shape)
posdata = np.stack(([['X11','X12','X13','X14'],['X21','X22','X23','X24']],[['NN11','NN12','NN13','NN14'],['NN21','NN22','NN23','NN24']],[['NNN11','NNN12','NNN13','NNN14'],['NNN21','NNN22','NNN23','NNN24']]),axis=2)
   
posdata=np.reshape(posdata,(2,4*3))
print(posdata)
posdata=np.reshape(posdata[0],(4,3))
print(posdata)

Instructions for updating:
non-resource variables are not supported in the long term
(2, 4)
[['X11' 'NN11' 'NNN11' 'X12' 'NN12' 'NNN12' 'X13' 'NN13' 'NNN13' 'X14'
  'NN14' 'NNN14']
 ['X21' 'NN21' 'NNN21' 'X22' 'NN22' 'NNN22' 'X23' 'NN23' 'NNN23' 'X24'
  'NN24' 'NNN24']]
[['X11' 'NN11' 'NNN11']
 ['X12' 'NN12' 'NNN12']
 ['X13' 'NN13' 'NNN13']
 ['X14' 'NN14' 'NNN14']]


In [2]:
def load_data(file_name,maxrow):
    print("read data from \"",file_name,"\"")
    
    xdata=np.loadtxt(file_name,max_rows=maxrow)
    L=(xdata.shape[1]-3)//9
    print(L)
    ydata=xdata[:,1]
    dxdata=xdata[:,xdata.shape[1]-1]
    
    posdata=xdata[:,2+4*L:2+5*L]
    NNdata=np.stack((xdata[:,2+L:2+2*L],xdata[:,2+3*L:2+4*L],xdata[:,2+5*L:2+6*L],xdata[:,2+7*L:2+8*L]),axis=2)
    NNNdata=np.stack((xdata[:,2:2+L],xdata[:,2+2*L:2+3*L],xdata[:,2+6*L:2+7*L],xdata[:,2+8*L:2+9*L]),axis=2)
    NNdata=np.mean(NNdata,axis=2)
    NNNdata=np.mean(NNNdata,axis=2)
    
    averagex=np.mean(xdata[:,2:xdata.shape[1]-1])
    print(averagex)
    gdata_append=0.25*(posdata+np.outer(dxdata,np.ones(L))-averagex)**4-averagex**2/2.0*(posdata+np.outer(dxdata,np.ones(L))-averagex)**2-0.25*(posdata-averagex)**4+averagex**2/2.0*(posdata-averagex)**2
    print(np.outer(dxdata,np.ones(L)).shape)
    gdata_append=np.mean(gdata_append,axis=1)
    zdata_append=np.sum((posdata+np.outer(dxdata,np.ones(L)))**2-posdata**2,axis=1)
    print(posdata.shape)
    posdata = np.stack((posdata,NNdata,NNNdata),axis=2)
   
    posdata=np.reshape(posdata,(maxrow,L*3))
    
    return posdata, dxdata, ydata, zdata_append, gdata_append


def load_data2(file_name,maxrow):
    print("read data from \"",file_name,"\"")
    
    xdata=np.loadtxt(file_name,max_rows=maxrow)
    L=(xdata.shape[1]-3)//9
    print(L)
    ydata=xdata[:,1]
    dxdata=xdata[:,xdata.shape[1]-1]
    
    posdata=xdata[:,2+8*L:2+9*L]
    NNdata=np.stack((xdata[:,2+L:2+2*L],xdata[:,2+3*L:2+4*L],xdata[:,2+4*L:2+5*L],xdata[:,2+6*L:2+7*L]),axis=2)
    NNNdata=np.stack((xdata[:,2:2+L],xdata[:,2+2*L:2+3*L],xdata[:,2+5*L:2+6*L],xdata[:,2+7*L:2+8*L]),axis=2)
    NNdata=np.mean(NNdata,axis=2)
    NNNdata=np.mean(NNNdata,axis=2)
    
    averagex=np.mean(xdata[:,2:xdata.shape[1]-1])
    print(averagex)
    gdata_append=0.25*(posdata+np.outer(dxdata,np.ones(L))-averagex)**4-averagex**2/2.0*(posdata+np.outer(dxdata,np.ones(L))-averagex)**2-0.25*(posdata-averagex)**4+averagex**2/2.0*(posdata-averagex)**2
    print(np.outer(dxdata,np.ones(L)).shape)
    gdata_append=np.mean(gdata_append,axis=1)
    zdata_append=np.sum((posdata+np.outer(dxdata,np.ones(L)))**2-posdata**2,axis=1)
    print(posdata.shape)
    posdata = np.stack((posdata,NNdata,NNNdata),axis=2)
   
    posdata=np.reshape(posdata,(maxrow,L*3))
    
    return posdata, dxdata, ydata, zdata_append, gdata_append



def scaled_data(xdata):
    xdata_mean=np.mean(xdata,axis=0)
    xdata_std=np.std(xdata,axis=0)
    xdata_scaled= (xdata-xdata_mean)/xdata_std
    return xdata_scaled, xdata_mean, xdata_std


def get_next_batch(x,y,start,end):
    x_batch=x[start:end,0:x.shape[1]-3]
    dx_batch=x[start:end,x.shape[1]-3]
    z_batch=x[start:end,x.shape[1]-2]
    g_batch=x[start:end,x.shape[1]-1]
    y_batch=y[start:end]
    return x_batch, dx_batch, z_batch, g_batch, y_batch


def weight_variable(name, shape):
    initer=tf.truncated_normal_initializer(stddev=0.02)
    return tf.get_variable('W_'+name, dtype=tf.float32,shape=shape,initializer=initer)

def bias_variable(name, shape):
    initial=tf.constant(0.0,shape=shape,dtype=tf.float32)
    return tf.get_variable('b_'+name,dtype=tf.float32,initializer=initial)

def create_new_conv_layer(input_data, deltax, num_filters, filter_shape, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], 1, num_filters]

    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                      name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 3, 1, 1],padding='VALID')

    # add the bias
    out_layer += bias
    
    # add the theta term
    in_dim=dx.get_shape()[1]
    theta = tf.Variable(tf.truncated_normal([in_dim,num_filters]), name=name+'_theta')
    out_layer +=tf.tensordot(tf.tensordot(tf.squeeze(dx),tf.ones([12],tf.float32),axes=0),theta,axes=0)

    # apply a sigmoid non-linear activation
    out_layer = tf.nn.sigmoid(out_layer)

    
    return out_layer, weights, bias, theta





def fc_layer(x,num_units,name,activation):
    in_dim=x.get_shape()[1]
    W=weight_variable(name,shape=[in_dim,num_units])
    b=bias_variable(name,[num_units])
    layer=tf.matmul(x,W)
    layer+=b
    if activation=='softplus':
        layer=tf.nn.softplus(layer)
    elif activation=='sigmoid':
        layer=tf.nn.sigmoid(layer)
    return layer, W, b

def fc_layer_final(x,g,z,num_units,name):
    in_dim=x.get_shape()[1]
    W1=weight_variable(name,shape=[in_dim,num_units])
    W2g=weight_variable(name+'g',shape=[1,num_units])
    W3z=weight_variable(name+'x',shape=[1,num_units])
    b=bias_variable(name,[num_units])
    
    layer=tf.matmul(x,W1)+tf.matmul(g,W2g)+tf.matmul(z,W3z)

    layer+=b
    return layer, W1, W2g, W3z, b

def save_data(file_name,name,x,matrix):
    out_file=open(file_name,'a')
    if matrix:
        out_file.write("%30s: [  %4d  ,  %4d  ]\n"%(name,x.shape[0], x.shape[1]))
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                out_file.write("%10.6f    " %(x[i,j]))
            out_file.write("\n")
    else:
        out_file.write("%s: [  %4d  ]\n"%(name,x.shape[0]))
        for i in range(x.shape[0]):
            out_file.write("%10.6f    "%(x[i]))
        out_file.write("\n")
    out_file.close()

In [3]:
# set parameters
#file_name="./traindata/build/globaltrain.dat"
file_name="./Desktop/block_train0003.dat"

maxrow=100000

# set hyper parameters
epochs=1000
batch_size=200
display_freq=50
learning_rate=0.001
L2_regularization=0.0

In [4]:
# read in data
#xdata, dxdata, ydata, zdata_append, gdata_append=load_data(file_name,maxrow)
xdata, dxdata, ydata, zdata_append, gdata_append=load_data2(file_name,maxrow)

# scaled data
xdata_scaled,        xdata_mean,        xdata_std        = scaled_data(xdata)
zdata_append_scaled, zdata_append_mean, zdata_append_std = scaled_data(zdata_append)
gdata_append_scaled, gdata_append_mean, gdata_append_std = scaled_data(gdata_append)
dxdata_scaled,       dxdata_mean,       dxdata_std       = scaled_data(dxdata)
xdata_combined=np.concatenate((xdata_scaled,np.array([dxdata_scaled]).T,np.array([zdata_append_scaled]).T,np.array([gdata_append_scaled]).T),axis=1)

# split data
x_train, x_test, y_train, y_test = train_test_split(xdata_combined, ydata, test_size=0.33, random_state=42)
L = (x_train.shape[1]-3)//3
print("Size of:")
print('x_train:\t{}'.format(x_train.shape))
print('y_train:\t{}'.format(y_train.shape))
print('x_test:\t{}'.format(x_test.shape))
print('y_test:\t{}'.format(y_test.shape))

read data from " ./Desktop/block_train0003.dat "
41
3.998107016961392
(100000, 41)
(100000, 41)
Size of:
x_train:	(67000, 126)
y_train:	(67000,)
x_test:	(33000, 126)
y_test:	(33000,)


In [14]:
print(gdata_append_std)

178.39236700513172


In [6]:
#create placeholders
x = tf.placeholder(tf.float32,shape=[None, x_train.shape[1]-3],name='X')
x_shaped = tf.reshape(x,[-1,L,3,1])
dx = tf.placeholder(tf.float32,shape=[None, 1] ,name='dX')
g = tf.placeholder(tf.float32,shape=[None, 1] ,name='g')
z = tf.placeholder(tf.float32,shape=[None, 1] ,name='z')
y = tf.placeholder(tf.float32,shape=[None,1] ,name='Y')

#define the network
convout, Wconv, bconv, thetaconv = create_new_conv_layer(x_shaped, dx, 10, [8,3], name="convlayer")
flattened = tf.reshape(convout, [-1, 10*((L-8)//3+1)])
fc1, W1, b1 = fc_layer(flattened, 20 , 'FC1' , 'sigmoid')

output, W2, W2g, W2z, b2 = fc_layer_final(fc1,g,z,1,'FC2')


loss=tf.reduce_mean(tf.squared_difference(output, y))
regularizer=tf.reduce_sum([tf.nn.l2_loss(W1), tf.nn.l2_loss(W2), tf.nn.l2_loss(W2g), tf.nn.l2_loss(W2z), tf.nn.l2_loss(Wconv), tf.nn.l2_loss(thetaconv)])
total_loss=tf.reduce_sum(loss+L2_regularization*regularizer)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate,name="Adam_op").minimize(total_loss)

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


In [7]:
# initializing all variables
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
num_tr_iter=int(len(y_train)/batch_size)
print('num_tr_iter = ', num_tr_iter)
store_loss_train_name='global_loss_train.dat'
sltn=open(store_loss_train_name,"w+")

for epoch in range(epochs):
    print('-----------------------------------------------')
    print('Training epoch : ', epoch)
    print('-----------------------------------------------')
    for iteration in range(num_tr_iter):
        start=iteration*batch_size
        end=(iteration+1)*batch_size
        x_batch,dx_batch,z_batch,g_batch,y_batch=get_next_batch(x_train,y_train,start,end)
        z_batch=np.expand_dims(z_batch,1)
        dx_batch=np.expand_dims(dx_batch,1)
        g_batch=np.expand_dims(g_batch,1)
        y_batch=np.expand_dims(y_batch,1)

        feed_dict_batch={x:x_batch,dx:dx_batch,g:g_batch,z:z_batch,y:y_batch}

        sess.run(optimizer,feed_dict=feed_dict_batch)

        if iteration%display_freq==0:
            loss_batch=sess.run(loss,feed_dict=feed_dict_batch)
            print("iter {0:3d}:\t Loss = {1:2f}".format(iteration,loss_batch))
            sltn.write("%6d    %15.12f\n" %(iteration+epoch*num_tr_iter, loss_batch))
sltn.close()



num_tr_iter =  335
-----------------------------------------------
Training epoch :  0
-----------------------------------------------
iter   0:	 Loss = 12.846389
iter  50:	 Loss = 7.513391
iter 100:	 Loss = 6.808902
iter 150:	 Loss = 7.009159
iter 200:	 Loss = 5.192062
iter 250:	 Loss = 6.698910
iter 300:	 Loss = 5.574524
-----------------------------------------------
Training epoch :  1
-----------------------------------------------
iter   0:	 Loss = 6.211347
iter  50:	 Loss = 4.942992
iter 100:	 Loss = 5.015193
iter 150:	 Loss = 5.237680
iter 200:	 Loss = 3.924478
iter 250:	 Loss = 4.421286
iter 300:	 Loss = 3.768161
-----------------------------------------------
Training epoch :  2
-----------------------------------------------
iter   0:	 Loss = 3.464777
iter  50:	 Loss = 2.736320
iter 100:	 Loss = 2.042642
iter 150:	 Loss = 2.066844
iter 200:	 Loss = 1.548259
iter 250:	 Loss = 1.266353
iter 300:	 Loss = 1.307016
-----------------------------------------------
Training epoch : 

iter 250:	 Loss = 0.079528
iter 300:	 Loss = 0.086455
-----------------------------------------------
Training epoch :  27
-----------------------------------------------
iter   0:	 Loss = 0.092298
iter  50:	 Loss = 0.106034
iter 100:	 Loss = 0.063330
iter 150:	 Loss = 0.096296
iter 200:	 Loss = 0.114768
iter 250:	 Loss = 0.078078
iter 300:	 Loss = 0.084372
-----------------------------------------------
Training epoch :  28
-----------------------------------------------
iter   0:	 Loss = 0.090645
iter  50:	 Loss = 0.103739
iter 100:	 Loss = 0.061840
iter 150:	 Loss = 0.094294
iter 200:	 Loss = 0.112735
iter 250:	 Loss = 0.076785
iter 300:	 Loss = 0.082523
-----------------------------------------------
Training epoch :  29
-----------------------------------------------
iter   0:	 Loss = 0.089310
iter  50:	 Loss = 0.101629
iter 100:	 Loss = 0.060551
iter 150:	 Loss = 0.092473
iter 200:	 Loss = 0.110890
iter 250:	 Loss = 0.075613
iter 300:	 Loss = 0.080871
----------------------------

iter 150:	 Loss = 0.072296
iter 200:	 Loss = 0.089696
iter 250:	 Loss = 0.064536
iter 300:	 Loss = 0.066046
-----------------------------------------------
Training epoch :  54
-----------------------------------------------
iter   0:	 Loss = 0.081013
iter  50:	 Loss = 0.084979
iter 100:	 Loss = 0.049769
iter 150:	 Loss = 0.071889
iter 200:	 Loss = 0.089328
iter 250:	 Loss = 0.064357
iter 300:	 Loss = 0.065671
-----------------------------------------------
Training epoch :  55
-----------------------------------------------
iter   0:	 Loss = 0.080970
iter  50:	 Loss = 0.084825
iter 100:	 Loss = 0.049581
iter 150:	 Loss = 0.071479
iter 200:	 Loss = 0.088985
iter 250:	 Loss = 0.064189
iter 300:	 Loss = 0.065269
-----------------------------------------------
Training epoch :  56
-----------------------------------------------
iter   0:	 Loss = 0.080943
iter  50:	 Loss = 0.084691
iter 100:	 Loss = 0.049386
iter 150:	 Loss = 0.071056
iter 200:	 Loss = 0.088663
iter 250:	 Loss = 0.064022
i

iter  50:	 Loss = 0.082361
iter 100:	 Loss = 0.044095
iter 150:	 Loss = 0.064754
iter 200:	 Loss = 0.081066
iter 250:	 Loss = 0.060530
iter 300:	 Loss = 0.059169
-----------------------------------------------
Training epoch :  81
-----------------------------------------------
iter   0:	 Loss = 0.081164
iter  50:	 Loss = 0.082281
iter 100:	 Loss = 0.043924
iter 150:	 Loss = 0.064618
iter 200:	 Loss = 0.080781
iter 250:	 Loss = 0.060470
iter 300:	 Loss = 0.059053
-----------------------------------------------
Training epoch :  82
-----------------------------------------------
iter   0:	 Loss = 0.081147
iter  50:	 Loss = 0.082202
iter 100:	 Loss = 0.043759
iter 150:	 Loss = 0.064486
iter 200:	 Loss = 0.080504
iter 250:	 Loss = 0.060412
iter 300:	 Loss = 0.058938
-----------------------------------------------
Training epoch :  83
-----------------------------------------------
iter   0:	 Loss = 0.081129
iter  50:	 Loss = 0.082124
iter 100:	 Loss = 0.043599
iter 150:	 Loss = 0.064359
i

iter  50:	 Loss = 0.080142
iter 100:	 Loss = 0.040879
iter 150:	 Loss = 0.061523
iter 200:	 Loss = 0.074961
iter 250:	 Loss = 0.059140
iter 300:	 Loss = 0.054739
-----------------------------------------------
Training epoch :  108
-----------------------------------------------
iter   0:	 Loss = 0.080007
iter  50:	 Loss = 0.080066
iter 100:	 Loss = 0.040838
iter 150:	 Loss = 0.061419
iter 200:	 Loss = 0.074733
iter 250:	 Loss = 0.059083
iter 300:	 Loss = 0.054443
-----------------------------------------------
Training epoch :  109
-----------------------------------------------
iter   0:	 Loss = 0.079918
iter  50:	 Loss = 0.079984
iter 100:	 Loss = 0.040809
iter 150:	 Loss = 0.061323
iter 200:	 Loss = 0.074501
iter 250:	 Loss = 0.059030
iter 300:	 Loss = 0.054143
-----------------------------------------------
Training epoch :  110
-----------------------------------------------
iter   0:	 Loss = 0.079826
iter  50:	 Loss = 0.079896
iter 100:	 Loss = 0.040790
iter 150:	 Loss = 0.06123

...

iter  50:	 Loss = 0.072586
iter 100:	 Loss = 0.038436
iter 150:	 Loss = 0.061876
iter 200:	 Loss = 0.062007
iter 250:	 Loss = 0.054243
iter 300:	 Loss = 0.042613
-----------------------------------------------
Training epoch :  243
-----------------------------------------------
iter   0:	 Loss = 0.071069
iter  50:	 Loss = 0.072559
iter 100:	 Loss = 0.038397
iter 150:	 Loss = 0.061872
iter 200:	 Loss = 0.061964
iter 250:	 Loss = 0.054218
iter 300:	 Loss = 0.042583
-----------------------------------------------
Training epoch :  244
-----------------------------------------------
iter   0:	 Loss = 0.070996
iter  50:	 Loss = 0.072531
iter 100:	 Loss = 0.038358
iter 150:	 Loss = 0.061868
iter 200:	 Loss = 0.061920
iter 250:	 Loss = 0.054193
iter 300:	 Loss = 0.042552
-----------------------------------------------
Training epoch :  245
-----------------------------------------------
iter   0:	 Loss = 0.070923
iter  50:	 Loss = 0.072503
iter 100:	 Loss = 0.038318
iter 150:	 Loss = 0.06186

iter  50:	 Loss = 0.071772
iter 100:	 Loss = 0.037360
iter 150:	 Loss = 0.061735
iter 200:	 Loss = 0.060920
iter 250:	 Loss = 0.053594
iter 300:	 Loss = 0.041887
-----------------------------------------------
Training epoch :  270
-----------------------------------------------
iter   0:	 Loss = 0.068977
iter  50:	 Loss = 0.071741
iter 100:	 Loss = 0.037321
iter 150:	 Loss = 0.061728
iter 200:	 Loss = 0.060884
iter 250:	 Loss = 0.053571
iter 300:	 Loss = 0.041864
-----------------------------------------------
Training epoch :  271
-----------------------------------------------
iter   0:	 Loss = 0.068897
iter  50:	 Loss = 0.071710
iter 100:	 Loss = 0.037282
iter 150:	 Loss = 0.061722
iter 200:	 Loss = 0.060848
iter 250:	 Loss = 0.053549
iter 300:	 Loss = 0.041842
-----------------------------------------------
Training epoch :  272
-----------------------------------------------
iter   0:	 Loss = 0.068817
iter  50:	 Loss = 0.071679
iter 100:	 Loss = 0.037243
iter 150:	 Loss = 0.06171

iter  50:	 Loss = 0.071015
iter 100:	 Loss = 0.036381
iter 150:	 Loss = 0.061585
iter 200:	 Loss = 0.060087
iter 250:	 Loss = 0.053016
iter 300:	 Loss = 0.041375
-----------------------------------------------
Training epoch :  297
-----------------------------------------------
iter   0:	 Loss = 0.066967
iter  50:	 Loss = 0.070991
iter 100:	 Loss = 0.036349
iter 150:	 Loss = 0.061581
iter 200:	 Loss = 0.060062
iter 250:	 Loss = 0.052996
iter 300:	 Loss = 0.041360
-----------------------------------------------
Training epoch :  298
-----------------------------------------------
iter   0:	 Loss = 0.066903
iter  50:	 Loss = 0.070967
iter 100:	 Loss = 0.036317
iter 150:	 Loss = 0.061577
iter 200:	 Loss = 0.060037
iter 250:	 Loss = 0.052977
iter 300:	 Loss = 0.041345
-----------------------------------------------
Training epoch :  299
-----------------------------------------------
iter   0:	 Loss = 0.066840
iter  50:	 Loss = 0.070944
iter 100:	 Loss = 0.036286
iter 150:	 Loss = 0.06157

iter  50:	 Loss = 0.070488
iter 100:	 Loss = 0.035611
iter 150:	 Loss = 0.061526
iter 200:	 Loss = 0.059494
iter 250:	 Loss = 0.052537
iter 300:	 Loss = 0.040999
-----------------------------------------------
Training epoch :  324
-----------------------------------------------
iter   0:	 Loss = 0.065572
iter  50:	 Loss = 0.070472
iter 100:	 Loss = 0.035586
iter 150:	 Loss = 0.061525
iter 200:	 Loss = 0.059474
iter 250:	 Loss = 0.052521
iter 300:	 Loss = 0.040985
-----------------------------------------------
Training epoch :  325
-----------------------------------------------
iter   0:	 Loss = 0.065533
iter  50:	 Loss = 0.070457
iter 100:	 Loss = 0.035562
iter 150:	 Loss = 0.061524
iter 200:	 Loss = 0.059454
iter 250:	 Loss = 0.052505
iter 300:	 Loss = 0.040972
-----------------------------------------------
Training epoch :  326
-----------------------------------------------
iter   0:	 Loss = 0.065494
iter  50:	 Loss = 0.070443
iter 100:	 Loss = 0.035537
iter 150:	 Loss = 0.06152

KeyboardInterrupt: 

In [ ]:
# plot training loss function
#matplotlib.figure.Figure()
xloss,yloss=np.loadtxt(store_loss_train_name,unpack=True)
plt.plot(xloss, yloss)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.show()

# save and plot y_predict_train, y_train
store_y_train_name='y_train.dat'
sytn=open(store_y_train_name,"w+")
np.random.seed(68721)
random_list=np.random.randint(len(y_train),size=2000)
x_batch=np.zeros([1000,x_train.shape[1]-3])
g_batch=np.zeros([1000,1])
dx_batch=np.zeros([1000,1])
z_batch=np.zeros([1000,1])
y_batch=np.zeros([1000,1])
for i in range(1000):
    x_batch[i,0:x_train.shape[1]-3]=x_train[random_list[i],0:x_train.shape[1]-3].T
    g_batch[i,0]=x_train[random_list[i],x_train.shape[1]-1]
    z_batch[i,0]=x_train[random_list[i],x_train.shape[1]-2]
    dx_batch[i,0]=x_train[random_list[i],x_train.shape[1]-3]
    y_batch[i,0]=y_train[random_list[i]]

feed_dict_batch={x:x_batch,dx:dx_batch,g:g_batch,z:z_batch,y:y_batch}
y_train_predict=sess.run(output,feed_dict=feed_dict_batch)
print(y_train_predict.shape)
for i in range(1000):
    sytn.write("%15.12f    %15.12f\n" %(y_batch[i,0], y_train_predict[i,0]))
sytn.close()

#matplotlib.figure.Figure()
y1, y2=np.loadtxt(store_y_train_name,unpack=True)
plt.plot(y1, y2,'ro')
plt.plot(y1,y1,'-b')
plt.xlabel('Y Value')
plt.ylabel('Y Prediction')
plt.show()